Accessing a Record's files
=====================

Sina can help you track files by associating a list of files with each record. These can be easily accessed and queried.

In [ ]:
import IPython.display as ipyd
import ipywidgets as widgets
import sina
import sina.utils
import tabulate
import six

database = sina.utils.get_example_path('noaa/data.sqlite')
print("Using database {}".format(database))

type_to_query = "obs"

print("Creating data access object factory.")
ds = sina.connect(database)
records = ds.records

print("Loading the data from the database.  This may take a while.")
all_ids = list(records.find_with_type(type_to_query, ids_only=True))

print("The data has loaded.  Now proceed to the next cell.")


"Get all files" dropdown
-----------------------------

Generate a dropdown selector for all available records. Display a table of all files associated with the selected record. Note that the default database only has one file per Record. This only shows the file(s)--we'll expand it into opening in the next set of cells.

In [ ]:
column_display_names = {"mimetype": "Mimetype", "uri": "URI", "tags": "Tags"}

select_widget = widgets.Dropdown(
    options=all_ids,
    value=all_ids[0],
    description='Record:',
    disabled=False)


def display_table(list_pos):
    """Displays a table populated by all files associated with a record"""
    all_files = records.get(all_ids[list_pos]).files
    # Files are stored in a dictionary, but tabulate expects a list.
    file_list = []
    for uri, file_data in six.iteritems(all_files):
        file_data["uri"] = uri
        file_list.append(file_data)
    tbl = tabulate.tabulate([column_display_names] + file_list, tablefmt='html')
    ipyd.display(ipyd.HTML(tbl))


def on_select(change):
    """Clears output, displays widget and file table"""
    ipyd.clear_output(wait=False)
    ipyd.display(select_widget)
    list_pos = change['owner'].index
    display_table(list_pos)


select_widget.observe(on_select)

ipyd.display(select_widget)
display_table(0)

Accessing file contents
=======================

As long as a file hasn't been moved, standard Python I/O procedure applies. We'll re-use the work of the previous cell, but now load the file contents instead.

In [ ]:
file_open_widget = widgets.Dropdown(
    options=all_ids,
    value=all_ids[0],
    description='Record:',
    disabled=False)


def open_file(list_pos):
    """Prints contents of all files associated with a record"""
    all_files = records.get(all_ids[list_pos]).files
    for uri, file_data in six.iteritems(all_files):
        try:
            with open(uri, "r") as infile:
                print("CONTENTS OF {}:\n{}".format(uri.upper(), "-" * 50))
                print(infile.read())
        except IOError:
            print("{} was not found"
                  .format(uri))


def on_select_file_open(change):
    """Clears output, displays widget and file contents"""
    ipyd.clear_output(wait=False)
    ipyd.display(file_open_widget)
    list_pos = change['owner'].index
    open_file(list_pos)


file_open_widget.observe(on_select_file_open)

ipyd.display(file_open_widget)
open_file(0)

Releasing Resources
-------------------------
Don't forget to release resources when you're all done!

In [ ]:
ds.close()